In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from tqdm import tqdm

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)), # try smaller things as details not that important
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

label_map = {
    'autumn': 0,
    'spring': 1,
    'summer': 2,
    'winter': 3
}

In color_type_clip:

- `clustered_by_color_type_manually_cleaned` was used for training
- `reference` was used for validation 
- `test` for testing

In [3]:
def images_to_tensors(path, transform):
    X = []  
    y = []  

    for label_name, label_id in label_map.items():
        folder = os.path.join(path, label_name)
        print(folder)
        if not os.path.isdir(folder):
            continue
        for filename in os.listdir(folder):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                new_path = os.path.join(folder, filename)
                try:
                    image = Image.open(new_path).convert("RGB")
                    image_tensor = transform(image)
                    X.append(image_tensor)
                    y.append(label_id)
                except Exception as e:
                    print(f"Error with file {new_path}: {e}")

    X = torch.stack(X)
    y = torch.tensor(y)
    return X, y

In [12]:
# Train and validation datasers load 
 
X_train, y_train = images_to_tensors('clustered_by_color_type_manually_cleaned/', train_transform)
X_val, y_val = images_to_tensors('reference/', val_transform)

clustered_by_color_type_manually_cleaned/autumn
clustered_by_color_type_manually_cleaned/spring
clustered_by_color_type_manually_cleaned/summer
clustered_by_color_type_manually_cleaned/winter
reference/autumn
reference/spring
reference/summer
reference/winter


In [18]:
# Test dataset load

X = []
y = []
for filename in os.listdir('tests'):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        path = os.path.join('tests', filename)
        try:
            image = Image.open(path).convert("RGB")
            image_tensor = val_transform(image)
            X.append(image_tensor)
        except Exception as e:
            print(f"Error with file {path}: {e}")

X_test = torch.stack(X)
df = pd.read_csv('true_labels.csv')
y_test = df['color_type'].map(label_map).values
y_test = torch.tensor(y_test, dtype=torch.long)

In [13]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
batch_size = 32
learning_rate = 0.001
num_epochs = 10
num_classes = 4

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x
    

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

In [15]:
weights = torch.tensor([1.0, 2.7, 3.1, 1.0], device=device)  # обратные пропорции
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total = 0 
    correct = 0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    acc = correct / total * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")


    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    val_acc = correct / total * 100
    print(f"Epoch {epoch}: Val Loss={val_loss/len(val_loader):.4f}, Val Acc={val_acc:.2f}")


100%|██████████| 150/150 [00:19<00:00,  7.56it/s]


Epoch [1/10], Loss: 189.8514, Accuracy: 40.91%
Epoch 0: Val Loss=0.9240, Val Acc=47.50


100%|██████████| 150/150 [00:19<00:00,  7.81it/s]


Epoch [2/10], Loss: 169.4059, Accuracy: 48.14%
Epoch 1: Val Loss=0.9596, Val Acc=50.00


100%|██████████| 150/150 [00:19<00:00,  7.83it/s]


...

Epoch [8/10], Loss: 67.3951, Accuracy: 78.81%
Epoch 7: Val Loss=2.0453, Val Acc=42.50


100%|██████████| 150/150 [00:19<00:00,  7.79it/s]


Epoch [9/10], Loss: 56.4693, Accuracy: 83.02%
Epoch 8: Val Loss=1.6838, Val Acc=50.00


100%|██████████| 150/150 [00:19<00:00,  7.78it/s]

Epoch [10/10], Loss: 43.5147, Accuracy: 86.03%
Epoch 9: Val Loss=2.9028, Val Acc=42.50


In [16]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Переводим модель в режим оценки
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)  # суммируем loss по батчу

            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy

In [19]:
test_model(model, DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False), criterion, device)

Test Loss: 5.4035, Test Accuracy: 0.2644


(5.403454265375247, 0.26436781609195403)

In [20]:
import joblib

joblib.dump(model, 'cnn_model.pkl')

['cnn_model.pkl']